# <center>SEGMENTATION DE LA CLIENTELE D'UNE ENTREPRISE.</center>

### OBJECTIF: 
       Apprendre a analyser un ensemble de données de transactions clients afin d'interprêter le comportement 
       des clients d'un fournisseur.

### TACHES: 
   * Analyse exploratoire des données
   * Analyse en composantes principales
   * Creation des clusters

## ANALYSE EXPLORATOIRE DES DONNEES

### IMPORTATION

In [ ]:
from warnings import filterwarnings
filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from scipy.stats import pearsonr
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

### DONNEES

In [ ]:
df = pd.read_csv("customers.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isna().sum()

In [ ]:
df["Channel"].value_counts()

In [ ]:
100 * df["Region"].value_counts(normalize = True)

In [ ]:
df = df.rename(columns = {"Delicassen" : "Delicatessen"})
df["Region"] = df["Region"].map({1: "Lisbon", 2: "Porto", 3: "Other"})

In [ ]:
colonnes = df.columns.tolist()
colonnes

In [ ]:
for colonne in colonnes:
    if colonne == "Channel" or colonne == "Region":
        sns.countplot(y = df[colonne], palette = 'cool').set_title(colonne)
        plt.show()
    else:
        sns.histplot(x = df[colonne], palette = 'cool').set_title(colonne)
        plt.show()

In [ ]:
sns.pairplot(df);

## ANALYSE EN COMPOSANTES PRINCIPALES

In [ ]:
data = df.drop(["Region", "Channel"], axis = 1)
data.head()

In [ ]:
#standardisation

scaler = StandardScaler()
data = scaler.fit_transform(data)
data

In [ ]:
#ACP LINEAIRE

pca = PCA(n_components = 2)
res_pca = pca.fit_transform(data)
res_pca

In [ ]:
plt.scatter(res_pca[:,0], res_pca[:,1])
plt.xlabel("composate_1")
plt.ylabel("composate_2")
plt.title("PC1 vs PC2");

In [ ]:
# coefficient de correlation de pearson

pearsonr(res_pca[:,0], res_pca[:,1])[0]

In [ ]:
# ACP non-lineaire

kpca = KernelPCA(n_components = 2, kernel = "rbf")
res_kpca_rbf = kpca.fit_transform(data)
res_kpca_rbf

In [ ]:
plt.scatter(res_kpca_rbf[:,0], res_kpca_rbf[:,1])
plt.xlabel("composate_1")
plt.ylabel("composate_2")
plt.title("PC1 vs PC2");

In [ ]:
# ajout des nouvelles colonnes au dataset

df["x_kpca_rbf"] = res_kpca_rbf[:,0]
df["y_kpca_rbf"] = res_kpca_rbf[:,1]

In [ ]:
df.head()

## CREATION DES CLUSTERS (KMEANS)

In [ ]:
# choisir le meilleur nombre de cluster

kmeans = KMeans()
visualizer = KElbowVisualizer(kmeans, k = (2, 12))
visualizer.fit(res_kpca_rbf)
visualizer.show();

In [ ]:
model = KMeans(n_clusters = 5)
model.fit(res_kpca_rbf)
cluster = model.predict(res_kpca_rbf)
cluster

In [ ]:
plt.scatter(res_kpca_rbf[:,0], res_kpca_rbf[:,1], c = cluster)
plt.xlabel("composate_1")
plt.ylabel("composate_2")
plt.title("PC1 vs PC2");

In [ ]:
df["cluster"] = cluster

In [ ]:
df.head()

In [ ]:
df["cluster"].unique()

### conclusion

In [ ]:
result = df.drop(["x_kpca_rbf", "y_kpca_rbf","Channel"], axis = 1).groupby(["cluster", "Region"]).mean()
result

In [ ]:
result.to_csv("result.csv")